# 1. Package loading

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import json
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from sklearn import preprocessing
import string
import langid
from textblob import TextBlob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import Counter
import langid
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/review-subjectivity-ordered/Review - Subjectivity.csv

/kaggle/input/review-polarity/Review - Polarity - 1.csv

/kaggle/input/review-polarity/Review - Polarity - 6.csv

/kaggle/input/review-polarity/Review - Polarity - 4.csv

/kaggle/input/review-polarity/Review - Polarity - 7.csv

/kaggle/input/review-polarity/Review - Polarity - 3.csv

/kaggle/input/review-polarity/Review - Polarity - 5.csv

/kaggle/input/review-polarity/Review - Polarity - 2.csv

/kaggle/input/non-english-review-id/Non English Review ID.csv

/kaggle/input/yelp-dataset/Dataset_User_Agreement.pdf

/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json

/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json

/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json

/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json

/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json


/opt/conda/lib/python3.10/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.

  warnings.warn("The twython library has not been installed. "


# 2.Data loading

## 2.1 Business data

In [2]:
with open('/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json') as data_file:
    data = [json.loads(line) for line in data_file]
df_business_final = pd.DataFrame(data)

df_business_final.drop(['address', 'hours'], axis=1, inplace=True)

In [3]:
def clean_business_data(df_cleaned):
    
    # address the missing values
    df_cleaned['attributes'].fillna('unknown', inplace=True)
    df_cleaned['categories'].fillna('未知', inplace=True)
    df_cleaned['review_count'].fillna(df_cleaned['review_count'].median(), inplace=True)
    df_cleaned['is_open'].fillna(0, inplace=True)

    # deal with missing postal code
    df_notnull = df_cleaned.dropna(subset=['postal_code'])
    for index, row in df_cleaned[df_cleaned['postal_code'].isnull()].iterrows():
        distances = df_notnull.apply(lambda x: abs(row['latitude'] - x['latitude']) + abs(row['longitude'] - x['longitude']), axis=1)
        nearest_index = distances.idxmin()
        df_cleaned.at[index, 'postal_code'] = df_notnull.at[nearest_index, 'postal_code']

    # check and address the outlier
    df_cleaned = df_cleaned[df_cleaned['latitude'].between(-90, 90) & df_cleaned['longitude'].between(-180, 180)]
    df_cleaned = df_cleaned[df_cleaned['stars'].between(0, 5)]
    
    # rename the column
    df_cleaned = df_cleaned.rename(columns = {'stars' : 'business_stars',
                                              'name' : 'business_name'})

    return df_cleaned

df_business_cleaned = clean_business_data(df_business_final)

### Filter the categories to restaurant

In [4]:
restaurant_ids = []

for i in range(0, len(df_business_cleaned)):
    string_list = df_business_cleaned['categories'][i].lower().split()
    if ('restaurants' in string_list) | ('restaurants,' in string_list):
        restaurant_ids.append(df_business_cleaned['business_id'][i])
# filter the business dataset by restaurant
df_business_restaurant = df_business_cleaned[df_business_cleaned['business_id'].isin(restaurant_ids)]

## 2.2 Review data

In [5]:
reviews = pd.read_json('/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json', lines=True, orient='columns', chunksize=10000)

df_review = []
for chunk in reviews:
  df_review.append(chunk)

df_reviews = pd.concat(df_review, ignore_index=True)
df_reviews['stars'] = df_reviews['stars'].astype(object)
df_reviews['text'] = df_reviews['text'].astype('str')

df_id = pd.read_csv('/kaggle/input/non-english-review-id/Non English Review ID.csv')
id_non_en = df_id['review_id']


### Detect and remove the non-English Review

In [ ]:
id_non_en = []
for i in range(0,700):
  for j in range(0, len(df_review[i])):
    text = df_review[i]['text']
    lang = langid.classify(text.iloc[j,])[0]
    if lang != 'en':
      id_non_en.append(df_review[i].iloc[j,]['review_id'])
df_non_en_review = pd.DataFrame({'review_id': id_non_en})
df_non_en_review.to_csv('Non English Review ID.csv')

# load the data
df_id = pd.read_csv('/kaggle/input/non-english-review-id/Non English Review ID.csv')
id_non_en = df_id['review_id']

# keep the English review only
df_en_review = df_reviews[~ df_reviews['review_id'].isin(id_non_en)]

In [ ]:
# check the duplication
df_review_unique = df_en_review.drop_duplicates(subset = ['review_id'], keep = 'first')
# remove the duplicate records with the same review contents to the same business (keep the first record)
df_review_cleaned = df_review_unique.drop_duplicates(subset = ['text', 'user_id', 'business_id'], keep = 'first')

# rename the columns
df_review_cleaned = df_review_cleaned.rename(columns = {'useful' : 'review_useful',
                                                        'funny' : 'review_funny',
                                                        'cool' : 'review_cool'})

# sort the reviews by time 
df_review_all = df_review_cleaned.sort_values(by = ['date'], ascending = False)

### Detect the sentiment of reviews: polarity and subjectivity

In [ ]:
# review polarity
polarity = []
sentiment = SentimentIntensityAnalyzer()
for i in range(0, len(df_review_cleaned)):
    text =  df_review_cleaned['text'].iloc[i]
    senti = sentiment.polarity_scores(text)
    polarity.append(senti['compound'])
df_polarity = pd.DataFrame({'polarity': polarity})
df_polarity.to_csv('Review - Polarity.csv') 

# add polarity
paths = ['/kaggle/input/review-polarity/Review - Polarity - 1.csv', '/kaggle/input/review-polarity/Review - Polarity - 2.csv',
         '/kaggle/input/review-polarity/Review - Polarity - 3.csv', '/kaggle/input/review-polarity/Review - Polarity - 4.csv',
         '/kaggle/input/review-polarity/Review - Polarity - 5.csv', '/kaggle/input/review-polarity/Review - Polarity - 6.csv',
         '/kaggle/input/review-polarity/Review - Polarity - 7.csv']
polarity = []
for i in range(0,7):
    polarity = polarity + pd.read_csv(paths[i])['polarity'].tolist()
    
df_review_all['rev_polarity'] = polarity

In [ ]:
# review subjectivity
subjectivity = []
for i in range(0, len(df_review_cleaned)):
    text =  df_review_cleaned['text'].iloc[i]
    subjectivity.append(TextBlob(text).sentiment.subjectivity)   
df_subjectivity = pd.DataFrame({'subjectivity': subjectivity})
df_subjectivity.to_csv('Review - Subjectivity.csv')

# add subjectivity
subjectivity = pd.read_csv('/kaggle/input/review-subjectivity-ordered/Review - Subjectivity.csv')['rev_subjectivity'].tolist()
df_review_all['rev_subjectivity'] = subjectivity

### Filter the categories to restaurant

In [6]:
# filter the review dataset to restaurants
df_review_restaurant = df_review_all[df_review_cleaned['business_id'].isin(restaurant_ids)]

# add the year of review
df_review_restaurant['year'] = df_review_restaurant['date'].dt.year

# add the text length to restaurant review
texts = df_review_restaurant['text'].reset_index()
text_length = []
for i in range(0, len(texts)):
  text_length.append(len(texts['text'][i].split()))
df_review_restaurant['review_length'] = text_length

In [7]:
# If need lastest 100W reviews
df_review_restaurant_100w = df_review_restaurant.iloc[0:1000000,]

## 2.3 User data

In [8]:
user = pd.read_json('/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json', lines=True, orient='columns', chunksize=10000)

df_user = []
for chunk in user:
  df_user.append(chunk)

df_user_final = pd.concat(df_user, ignore_index=True)

In [9]:
def clean_user_data(df):

    # delete the duplicated rows
    df_cleaned=df.drop_duplicates()

    # switch the column 'yelping_since' to date type
    df_cleaned['yelping_since'] = pd.to_datetime(df_cleaned['yelping_since'])

    # merge compliment columns
    compliment_columns = [col for col in df_cleaned.columns if col.startswith('compliment_')]
    df_cleaned['compliment_total'] = df_cleaned[compliment_columns].sum(axis=1)

    # detect and remove the outlier
    df_cleaned = df_cleaned[df_cleaned['average_stars'].between(0, 5)]

    # count the total number of friends 
    df_cleaned['friends_count'] = df_cleaned['friends'].fillna('[]').str.count(',') + 1
    df_cleaned['friends_count'] = df_cleaned.apply(lambda x: 0 if x['friends'] == '[]' else x['friends_count'], axis=1)

    # count the total number of elite years
    #df_cleaned['elite'] = df_cleaned['elite'].replace('', 'No')
    df_cleaned['elites_count'] = df_cleaned['elite'].str.count(',') + 1
    df_cleaned['elites_count'] = df_cleaned.apply(lambda x: 0 if x['elite'] == '' else x['elites_count'], axis=1)
    
    # rename the columns 
    df_cleaned = df_cleaned.rename(columns = {'useful' : 'user_useful',
                                              'funny' : 'user_funny',
                                              'cool' : 'user_cool',
                                              'name' : 'user_name'})
    
    return df_cleaned

df_user_cleaned=clean_user_data(df_user_final)

In [10]:
# filter the user dataset by restaurant
rev_num = df_review_restaurant.groupby(['user_id']).agg({'review_id':'count', 'stars':'mean', 'rev_polarity': 'mean'}).reset_index()
onlyRestaurantReviewCount = rev_num[['user_id','review_id','stars','rev_polarity']].rename(columns = {'review_id':'review_count','stars':'average_stars'})
# define active / non-active user based on the restaurant review
onlyRestaurantReviewCount['user_activity'] = 'Non-active User'
onlyRestaurantReviewCount.loc[onlyRestaurantReviewCount['review_count'] > 1, 'user_activity'] = 'Active User'
userIdRestaurant = df_review_restaurant['user_id'].unique().tolist()
df_user_restaurant = df_user_cleaned[df_user_cleaned['user_id'].isin(userIdRestaurant)]
df_user_restaurant= df_user_restaurant.drop(['review_count', 'average_stars'], axis=1)
##df_user_restaurant = df_user_restaurant[['user_id','user_name','yelping_since', 'friends_count','elites_count', 'user_type', 'user_famous', 'elite_status']]
df_user_restaurant = df_user_restaurant.merge(onlyRestaurantReviewCount, on = ['user_id'], how = 'left')

## 2.4 Merge three datasets

In [11]:
# merge the three datasets
df_all = df_review_restaurant.merge(df_business_restaurant, on = ['business_id'], how = 'left').merge(df_user_restaurant, on = ['user_id'], how = 'left') 

In [12]:
df_100w = df_all.iloc[0:1000000, ]

In [16]:
df_100w.groupby(['stars', 'user_activity']).agg({'review_length': 'mean'})

review_length
stars user_activity                 
1     Active User         107.691146
      Non-active User     102.981694
2     Active User         118.674185
      Non-active User     105.265649
3     Active User         115.976773
      Non-active User      95.462004
4     Active User         103.021282
      Non-active User      69.928466
5     Active User          77.959408
      Non-active User      56.297571

## 2.5 Text Preprocessing and Word Embedding

In [41]:
import re
import spacy
from time import time
import logging 
import multiprocessing
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer

In [42]:
df_sub = df_all.iloc[0:25000, ]

In [43]:
# text preprocessing
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed
def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long, the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt).split()
    
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df_sub['text'])
txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000)]
df_nlp_w = pd.DataFrame({'review_id': df_sub['review_id'],
                         'nlp_clean_review': txt})
df_nlp_w = df_nlp_w.dropna(subset = ['nlp_clean_review']).drop_duplicates(['nlp_clean_review'])

### Word2Vec

In [44]:
# Word embedding 
# sg: (default 0 or CBOW) The training algorithm, either CBOW (0) or skip gram (1).
# min_count: The minimum count of words to consider when training the model; words with an occurrence less than this count will be ignored.
# vector_size: (default 100) The number of dimensions of the embedding, e.g. the length of the dense vector to represent each token (word).
# window: (default 5) The maximum distance between a target word and words around the target word.

w2v = Word2Vec(df_nlp_w['nlp_clean_review'], window = 5, vector_size=200, min_count = 10, sg = 0)

# get sentence vector by calculate the average word vectors and take the weight from tf-idf
data = df_nlp_w['nlp_clean_review']
avg_data = []

for row in data:       
    vec = np.zeros(200)
    count = 0
    for word in row:
        try:
            if word in dict_tfidf:
                vec += w2v.wv[word] * dict_tfidf[word]
            else: vec += w2v.wv[word]
            count += 1
        except:
            pass
    avg_data.append(vec/count)  

df_nlp_v = pd.DataFrame(avg_data)
df_nlp_v['review_id'] = df_nlp_w['review_id']

In [45]:
merge word2vec to main dataset
df_all = df_all.merge(df_nlp_v, on = ['review_id'], how = 'inner')

### Pre-trained GloVe

In [46]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [47]:
#df_100w = pd.read_csv('df_100w_all_noWord2Vec_withText.csv', index_col = [0])

In [48]:
# Generate a cleaned reviews array from original review texts
def clean_document(doco):
    punctuation = string.punctuation + '\n\n';
    punc_replace = ''.join([' ' for s in punctuation]);
    doco_clean = doco.replace('-', ' ');
    doco_alphas = re.sub(r'\W +', '', doco_clean)
    trans_table = str.maketrans(punctuation, punc_replace);
    doco_clean = ' '.join([word.translate(trans_table) for word in doco_alphas.split(' ')]);
    doco_clean = doco_clean.split(' ');
    doco_clean = [word.lower() for word in doco_clean if len(word) > 0];
    
    return doco_clean;

# Generate a cleaned reviews array from original review texts
review_cleans = [clean_document(row) for row in df_sub['text']];  ### change to df_100w
sentences = [' '.join(r) for r in review_cleans]

tokenizer = Tokenizer();
tokenizer.fit_on_texts(sentences);
sequence_dict = tokenizer.word_index;

In [49]:
embeddings_index = dict();
with open('/kaggle/input/pre-trained-glove/glove.6B.200d.txt') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

In [50]:
# using pre-trained GloVe
# get sentence vector by calculate the average word vectors and take the weight from tf-idf 
data = df_nlp_w['nlp_clean_review']
avg_data = []

for row in data:       
    vec = np.zeros(200)
    count = 0
    for word in row:
        try:
            if word in dict_tfidf:
                vec += embeddings_index.get(word) * dict_tfidf[word]
            else: vec += embeddings_index.get(word)
            count += 1
        except:
            pass
    avg_data.append(vec/count)  

df_nlp_glove = pd.DataFrame(avg_data)
df_nlp_glove['review_id'] = df_nlp_w['review_id'].tolist()

## 2.6 Grouping users by single-review and multi-review

In [ ]:
# multi-review users
df_active = df_all[df_all['user_activity'] == 'Active User']

df_non_active = df_all[df_all['user_activity'] == 'Non-active User']

# 3. Feature Engineering

In [ ]:
import featuretools as ft
dataframes = {
    "review": (df_review_cleaned2, "review_id"),
    "business": (df_business_cleaned, "business_id"),
    "user": (df_user_cleaned, "user_id"),
}

relationships = [
    ("business", "business_id", "review", "business_id"),
    ("user", "user_id", "review", "user_id"),
]
feature_matrix, features_defs = ft.dfs(
    dataframes=dataframes,
    relationships=relationships,
    target_dataframe_name="review",
)
feature_matrix.head()